In [ ]:
import numpy as np
import torch

from data import JSRTDataModule
from torch.utils.data import DataLoader

from models.mae import ViTAE
from slot_attention import SlotAttention

import matplotlib.pyplot as plt
import matplotlib